In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import requests
import pandas as pd
import numpy as np

Nuestros jefes están especialmente interesados en los datos de generación de las energías renovables y no renovables a nivel nacional. 

En concreto, nos piden sacar toda la información para las fechas de 1 de Enero del 2011 hasta el 31 de Diciembre del 2022. De toda la información que tenemos de generación queremos la evolucion de renovable y no renovable. 

Además deberemos incluir una columna nueva identificando si el dato corresponde a energía renovable o no renovable. 


NOTA: nos piden que el código debe estar en una función. Al final deberemos tener un dataframe

Creamos un dataframe vacío para añadir los valores.

In [62]:
def renovables(inicio, fin):
    datos_energia = pd.DataFrame()
    for i in range(inicio, fin):
        url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={i}-01-01T00:00&end_date={i}-12-31T23:59&time_trunc=year"
        response = requests.get(url=url)
        datos = response.json()

        for e in range(len(datos["included"])):
            x = pd.DataFrame(datos["included"][e]["attributes"]["values"])
            x["tipo_energia"] = datos["included"][e]["attributes"]["title"]

            datos_energia = pd.concat([datos_energia, x], axis=0, ignore_index = True)

    return datos_energia

In [63]:
datos_tipos_energia = renovables(2011, 2023)

In [65]:
datos_tipos_energia.sample(5)

,value,percentage,datetime,tipo_energia
1,1.926868e+08,0.689759,2011-01-01T00:00:00.000+01:00,No renovable
4,1.097577e+08,0.401622,2013-01-01T00:00:00.000+01:00,Renovable
11,1.607465e+08,0.613921,2016-01-01T00:00:00.000+01:00,No renovable
5,1.635286e+08,0.598378,2013-01-01T00:00:00.000+01:00,No renovable
22,1.165831e+08,0.421921,2022-01-01T00:00:00.000+01:00,Renovable


In [10]:
df_renovables = datos_tipos_energia[datos_tipos_energia["tipo_energia"] == "Renovable"]
df_no_renovables = datos_tipos_energia[datos_tipos_energia["tipo_energia"] == "No renovable"]

In [14]:
display(df_renovables.sample(2))
display(df_no_renovables.sample(2))

,value,percentage,datetime,tipo_energia
2450,250655.3665,0.326260,2014-09-16T00:00:00.000+02:00,Renovable
3096,201968.4190,0.278284,2015-06-24T00:00:00.000+02:00,Renovable


,value,percentage,datetime,tipo_energia
6233,528517.7785,0.594427,2019-01-25T00:00:00.000+01:00,No renovable
3591,324377.5630,0.525998,2015-11-01T00:00:00.000+01:00,No renovable


Sacaremos la misma información que en el ejericio anterior, pero en este caso por CCAA. 

Para esto necesitaremos los códigos de cada una de las comunidades autónomas que encontraréis en el siguiente diccionario:

In [15]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [54]:
def comunidades(inicio, fin):
    datos_energia = pd.DataFrame()
    for i in range(inicio, fin):
        for k,v in cod_comunidades.items():
            url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={i}-01-01T00:00&end_date={i}-12-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={v}"
            response = requests.get(url=url)
            datos = response.json()

            for e in range(len(datos["included"])):
                x = pd.DataFrame(datos["included"][e]["attributes"]["values"])
                x["tipo_energia"] = datos["included"][e]["attributes"]["title"]
                x["comunidad"] = k
                x["cod_comunidad"] = v

                datos_energia = pd.concat([datos_energia, x], axis=0, ignore_index = True)

    return datos_energia

In [58]:
energia_comunidades = comunidades(2011, 2023)

Guardad los csv ya que mañana los seguiremos usando en los siguientes pairs

In [60]:
energia_comunidades.sample(5)

,value,percentage,datetime,tipo_energia,comunidad,cod_comunidad
336,1.440303e+07,0.514666,2020-01-01T00:00:00.000+01:00,Renovable,Andalucía,4
325,2.737325e+05,0.061925,2019-01-01T00:00:00.000+01:00,Renovable,Illes Balears,8743
263,2.251321e+07,0.657006,2018-01-01T00:00:00.000+01:00,No renovable,Andalucía,4
13,6.381742e+06,0.151747,2011-01-01T00:00:00.000+01:00,Renovable,Cataluña,9
216,7.014236e+05,0.080217,2016-01-01T00:00:00.000+01:00,Renovable,Canarias,8742


In [61]:
energia_comunidades.to_csv("datos/energia_comunidades.csv")

In [66]:
datos_tipos_energia.to_csv("datos/tipos_energia.csv")